In [2]:
using Statistics
#plotly()
# using Plots

In [13]:
using DistributedArrays

In [24]:
# addprocs(
#     [("alanedelman@anubis.juliacomputing.io", 60)],
#     exename = "/home/andreasnoack/julia06/bin/julia",
#     exeflags = "--compilecache=no",
#     dir = "/home/alanedelman"
# )
nworkers()

LoadError: UndefVarError: nprocs not defined

LoadError: UndefVarError: numprocs not defined

In [3]:
# Generates random data distributed proportionally to f(x) on [a,b]

In [25]:
#@everywhere begin

#using DistributedArrays

function jump(f, x, a = -1.0, b = 1.0)
    y = x + 0.1 * (rand()-.5)   
    rand() < f(y)/f(x) && a<y<b ? y  : x
end


function walk(f, n=10_000_000, x = 0.0, a = -1.0, b = 1.0)
 data = Float64[]
  h = Histogram(a:.01:b ,:left)
  for i=1:n-1
    x = jump(f,x)
    push!(h,x)
    push!(data,x)
  end
  return h,data 
end
    
#end

walk (generic function with 5 methods)

In [27]:
 f(x)=x.^2

f (generic function with 1 method)

In [16]:
a = distribute( randn(41))

41-element DArray{Float64, 1, Vector{Float64}}:
  0.48059753677424
 -1.011235451361166
  0.47858939128330025
 -0.12142050567236685
  0.18919899028500978
  1.4920766107065266
 -0.44049596202597235
  0.38295503157962973
 -1.247817836215618
  0.9327421959145253
  0.2516358499023996
 -0.46198715498096127
 -1.1613846325742114
  ⋮
 -0.6633299492226804
  0.28634482716582244
 -1.654830874902433
 -0.11840486290184836
 -1.4842707676369016
 -0.3559367618043355
  0.9610698385277565
  0.4641761714073894
  0.4074521957546799
  0.2857089958151928
 -0.16175895100332166
 -1.0149383650219297

In [5]:
using Interact

WebIO._IJuliaInit()

In [8]:
using Colors

In [9]:
d = distinguishable_colors(100)

In [17]:
#@manipulate for n=1:100
 # a = distribute( randn(n))
 # g(x) =length(x[1])
 # d[g.(a.indexes)]
#end

n=20

d[hcat( [g.(distribute( randn(n)).indexes) for n=10:100]...)]



LoadError: type DArray has no field indexes

In [18]:
@DArray [randn() for i=1:10]

10-element DArray{Float64, 1, Vector{Float64}}:
  2.3028811023627926
 -1.0951306324773535
 -1.3685849671471855
  0.2251475275723136
 -1.3943106868099244
  0.7721199949368848
  0.1543932921164958
  1.0846141550075206
  0.519981870058519
 -0.785213836851063

In [26]:
 [walk(f,10) for i=1:10]

LoadError: UndefVarError: f not defined

In [53]:
Array(D)

100-element Array{Tuple{StatsBase.Histogram{Int64,1,Tuple{StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}}},Array{Float64,1}},1}:
 (StatsBase.Histogram{Int64,1,Tuple{StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}}}
edges:
  -1.0:0.01:1.0
weights: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
closed: left
isdensity: false, [0.0383035, 0.0383035, 0.0383035, 0.0396647, 0.0396647, 0.0396647, 0.0469012, 0.0469012, 0.0675381])         
 (StatsBase.Histogram{Int64,1,Tuple{StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}}}
edges:
  -1.0:0.01:1.0
weights: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
closed: left
isdensity: false, [-0.0329991, -0.0329991, -0.0352913, -0.0352913, -0.0352913, -0.0157906, 0.0334537, 0.0334537, 0.0625616])   
 (StatsBase.Histogram{Int64,1,Tuple{StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}}}
edges:
  -

In [48]:
D.indexes

10-element Array{Tuple{UnitRange{Int64}},1}:
 (1:10,)  
 (11:20,) 
 (21:30,) 
 (31:40,) 
 (41:50,) 
 (51:60,) 
 (61:70,) 
 (71:80,) 
 (81:90,) 
 (91:100,)

In [30]:
h, data = walk(f)
data

LoadError: UndefVarError: Histogram not defined

In [29]:
@everywhere begin
    f(x) =  abs(x)
end

mid(r) = r[1:length(r) - 1] + 0.5 * step(r)
@time h = walk(f)


plot(h)
x = mid(h.edges[1])
y = h.weights 
plot!(x,y,label="observed")
Y = f.(x)
plot!(x, (y'Y)/(Y'Y)*Y, color="red", label="predicted", width=2)

LoadError: LoadError: UndefVarError: @everywhere not defined
in expression starting at In[29]:1

In [28]:
 h, data = @parallel (x,y)->(merge(x[1],y[1]), vcat(x[2],y[2])) for p = 1:nworkers()
  walk(f,100)
end

#=
plot(h,label=false)
x = mid(h.edges[1])
y = h.weights 
plot!(x,y,label="observed")
Y = f.(x)
plot!(x, (y'Y)/(Y'Y)*Y, color="red", label="predicted", width=2)
=#
data

LoadError: LoadError: UndefVarError: @parallel not defined
in expression starting at In[28]:1

In [ ]:
@parallel (vcat) for p=1:nworkers()
   myid()
end

In [ ]:
function foo(x)
    34
    45
    return
end

In [ ]:
x = nothing
typeof(x)

In [ ]:
sum(h.weights)

In [ ]:
length(h.edges[1])

In [ ]:
merge(h,h)

In [ ]:
nworkers()

In [ ]:
h2 = Histogram(-1:.01:1 ,:left)

In [ ]:
vcat( rand(3), rand(4))

In [ ]:
x =[rand(3,3) for i=1:100_000];

In [ ]:
length(x)

In [ ]:
x = rand(3,3,100_000);

In [ ]:
 # @fetchfrom 3 localpart(dD) 

In [54]:
a = drand(10)

10-element DistributedArrays.DArray{Float64,1,Array{Float64,1}}:
 0.105518 
 0.758881 
 0.408368 
 0.254824 
 0.822384 
 0.67095  
 0.0221236
 0.679513 
 0.37678  
 0.406047 

In [56]:
a = drand(100,100)
a.indexes

2×5 Array{Tuple{UnitRange{Int64},UnitRange{Int64}},2}:
 (1:50, 1:20)    (1:50, 21:40)    …  (1:50, 61:80)    (1:50, 81:100)  
 (51:100, 1:20)  (51:100, 21:40)     (51:100, 61:80)  (51:100, 81:100)

In [58]:
?drandn

search: drandn drand



```
 drandn(dims, ...)
```

Construct a distributed normal random array. Trailing arguments are the same as those accepted by `DArray`.


In [59]:
@everywhere importall DistributedArrays.SPMD

d_in=d=DArray(I->fill(myid(), (map(length,I)...)), (nworkers(), 2), workers(), [nworkers(),1])
d_out=ddata()

# define the function everywhere
@everywhere function foo_spmd(d_in, d_out, n)
    pids = sort(vec(procs(d_in)))
    pididx = findfirst(pids, myid())
    mylp = d_in[:L]
    localsum = 0

    # Have each worker exchange data with its neighbors
    n_pididx = pididx+1 > length(pids) ? 1 : pididx+1
    p_pididx = pididx-1 < 1 ? length(pids) : pididx-1

    for i in 1:n
        sendto(pids[n_pididx], mylp[2])
        sendto(pids[p_pididx], mylp[1])

        mylp[2] = recvfrom(pids[p_pididx])
        mylp[1] = recvfrom(pids[n_pididx])

        barrier(;pids=pids)
        localsum = localsum + mylp[1] + mylp[2]
    end

    # finally store the sum in d_out
    d_out[:L] = localsum
end

# run foo_spmd on all workers
spmd(foo_spmd, d_in, d_out, 10)

# print values of d_in and d_out after the run
println(d_in)
println(d_out)

LoadError: [91m[91m[91mBoundsError: attempt to access 0×0 Array{Int64,2} at index [2][39m
foo_spmd(::DistributedArrays.DArray{Int64,2,Array{Int64,2}}, ::DistributedArrays.DArray{Any,1,Any}, ::Int64) at ./In[59]:18
spmd_local(::DistributedArrays.SPMD.##25#27{#foo_spmd,Tuple{DistributedArrays.DArray{Int64,2,Array{Int64,2}},DistributedArrays.DArray{Any,1,Any},Int64}}, ::Tuple{Int64,Int64}, ::Bool) at /Users/alanedelman/.julia/v0.6/DistributedArrays/src/spmd.jl:228
(::Base.Distributed.##135#136{DistributedArrays.SPMD.#spmd_local,Tuple{DistributedArrays.SPMD.##25#27{#foo_spmd,Tuple{DistributedArrays.DArray{Int64,2,Array{Int64,2}},DistributedArrays.DArray{Any,1,Any},Int64}},Tuple{Int64,Int64},Bool},Array{Any,1}})() at ./distributed/remotecall.jl:314
run_work_thunk(::Base.Distributed.##135#136{DistributedArrays.SPMD.#spmd_local,Tuple{DistributedArrays.SPMD.##25#27{#foo_spmd,Tuple{DistributedArrays.DArray{Int64,2,Array{Int64,2}},DistributedArrays.DArray{Any,1,Any},Int64}},Tuple{Int64,Int64